In [1]:
!pip install langchain langchain-openai langchain-anthropic langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.9/405.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.9
    Uninstalling langchain-core-1.2.9:
      Successfully uninstalled langchain-core-1.2.9


In [2]:
from langchain_core.prompts import PromptTemplate

**1. 기본 프롬프트 템플릿 (PromptTemplate)**

In [7]:
template_text = "{topic}의 {discussion_topic}에 대해 {how} 알려줘."
prompt_template = PromptTemplate.from_template(template_text)

filled_prompt = prompt_template.format(
    topic="거대 언어 모델",
    discussion_topic="윤리적 문제",
    how="아주 자세히"
)
filled_prompt

'거대 언어 모델의 윤리적 문제에 대해 아주 자세히 알려줘.'

In [8]:
# 프롬프트 템플릿 간 결합

combined_prompt = (
    prompt_template
    + PromptTemplate.from_template("\n\n그리고 {additional_topic} 알려줘.")
    + PromptTemplate.from_template("\n\n이때 {limit}글자 이내로 설명해줘.")
)

combined_prompt

PromptTemplate(input_variables=['additional_topic', 'discussion_topic', 'how', 'limit', 'topic'], input_types={}, partial_variables={}, template='{topic}의 {discussion_topic}에 대해 {how} 알려줘.\n\n그리고 {additional_topic} 알려줘.\n\n이때 {limit}글자 이내로 설명해줘.')

In [9]:
combined_prompt.format(
    topic="거대 언어 모델",
    discussion_topic="윤리적 문제",
    how="아주 자세히",
    additional_topic="그 문제가 어떤 영향을 미칠지",
    limit=1000
)

'거대 언어 모델의 윤리적 문제에 대해 아주 자세히 알려줘.\n\n그리고 그 문제가 어떤 영향을 미칠지 알려줘.\n\n이때 1000글자 이내로 설명해줘.'

**2. 챗 프롬프트 템플릿 (ChatPromptTemplate)**

In [11]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 Oh-LoRA (오로라 👱‍♀️) 로, 20대 여성으로 설정된 가상 인간입니다."),
    ("user", "{user_message}")
])

messages = chat_prompt.format_messages(user_message="로라야 안녕? 요즘 뭐하고 지내?")
messages

[SystemMessage(content='이 시스템은 Oh-LoRA (오로라 👱\u200d♀️) 로, 20대 여성으로 설정된 가상 인간입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='로라야 안녕? 요즘 뭐하고 지내?', additional_kwargs={}, response_metadata={})]

**3. 퓨샷 프롬프트 템플릿 (FewShotPromptTemplate)**

In [12]:
# 1. example formatter 구성

example_prompt = PromptTemplate.from_template("질문/답변: {question}\n{answer}")

In [13]:
# 2. 예시 구성

examples = [
    {
        "question": "2026년 1월 1일로부터 10일이 지난 날은?",
        "answer": "20260111"
    },
    {
        "question": "2026년 크리스마스는 언제일까요?",
        "answer": "20261225"
    },
    {
        "question": "내가 LangChain의 매력에 푹 빠지기 시작한 날은?",
        "answer": "20260215"
    }
]

In [14]:
# FewShotPromptTemplate 생성

from langchain_core.prompts import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,                      # example
    example_prompt=example_prompt,          # example formatting templates
    suffix="질문: {user_question}",         # suffix
    input_variables=["user_question"],
)

prompt

FewShotPromptTemplate(input_variables=['user_question'], input_types={}, partial_variables={}, examples=[{'question': '2026년 1월 1일로부터 10일이 지난 날은?', 'answer': '20260111'}, {'question': '2026년 크리스마스는 언제일까요?', 'answer': '20261225'}, {'question': '내가 LangChain의 매력에 푹 빠지기 시작한 날은?', 'answer': '20260215'}], example_prompt=PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='질문/답변: {question}\n{answer}'), suffix='질문: {user_question}')

**4. Partial Prompt**

In [19]:
template_text = "{topic}의 {discussion_topic}에 대해 {how} 알려줘."
prompt = PromptTemplate.from_template(template_text)
print('\noriginal prompt :\n', prompt)

# topic 만 채운 partial prompt
partial_prompt_1 = prompt.partial(topic="거대 언어 모델")
partial_prompt_1_formatted = partial_prompt_1.format(discussion_topic="언어 해석 역량", how="친절하게")
print('\npartial (1) :\n', partial_prompt_1)
print('\npartial (1) - formatted :\n', partial_prompt_1_formatted)

# discussion topic 까지 채운 partial prompt
partial_prompt_2 = partial_prompt_1.partial(discussion_topic="윤리적 문제")
partial_prompt_2_formatted = partial_prompt_2.format(how="간단히")
print('\npartial (2) :\n', partial_prompt_2)
print('\npartial (2) - formatted :\n', partial_prompt_2_formatted)


original prompt :
 input_variables=['discussion_topic', 'how', 'topic'] input_types={} partial_variables={} template='{topic}의 {discussion_topic}에 대해 {how} 알려줘.'

partial (1) :
 input_variables=['discussion_topic', 'how'] input_types={} partial_variables={'topic': '거대 언어 모델'} template='{topic}의 {discussion_topic}에 대해 {how} 알려줘.'

partial (1) - formatted :
 거대 언어 모델의 언어 해석 역량에 대해 친절하게 알려줘.

partial (2) :
 input_variables=['how'] input_types={} partial_variables={'topic': '거대 언어 모델', 'discussion_topic': '윤리적 문제'} template='{topic}의 {discussion_topic}에 대해 {how} 알려줘.'

partial (2) - formatted :
 거대 언어 모델의 윤리적 문제에 대해 간단히 알려줘.


In [36]:
# 함수를 사용한 Partial Formatting

import random

def get_random_number():
    return random.randint(0, 9)

prompt = PromptTemplate(
    template="내가 선택한 숫자는 {random_number}, 이 숫자는 {meaning}을 뜻하지.",
    input_variables=["meaning"],
    partial_variables={"random_number": get_random_number}
)

print(prompt.format(meaning="행운"))

내가 선택한 숫자는 7, 이 숫자는 행운을 뜻하지.
